## Object references, mutability and recycling


### Variables are not boxes


In [1]:
a = [1, 2, 3]
b = a 
a.append(4)
b

[1, 2, 3, 4]

In [2]:
class Gizmo:
    def __init__(self):
        print('Gizmo id: {}'.format(id(self)))
        

In [3]:
x = Gizmo()

Gizmo id: 140394433461216


In [4]:
y = Gizmo() * 10

Gizmo id: 140394432045632


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

In [5]:
dir()

['Gizmo',
 'In',
 'Out',
 '_',
 '_1',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_i4',
 '_i5',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'a',
 'b',
 'exit',
 'get_ipython',
 'quit',
 'x']

In [7]:
# 体会不同

a = [1, 2, 3, 4]
b = [1, 2, 3, 4]

a == b

True

In [8]:
a is b

False

In [9]:
c = a
print(a == c)
print(a is c)

True
True


In [10]:
d = 1
d is 1

True

### tuple

In [11]:
a = (1, 2, [3, 4])
b = (1, 2, [3, 4])
print(a == b)
print(id(a[-1]))
a[-1].append(5)
print(id(a[-1]))
print(a == b)

True
140394432720840
140394432720840
False


### [:] vs copy


In [13]:
import copy
a = [1, 2, 3, [4, 5]]
b = a[:]
c = copy.copy(a)

print('b is {}'.format(b))
print('c is {}'.format(c))
print('b is a: {}'.format(b is a))
print('c is a: {}'.format(a is c))
print('b is c: {}'.format(b is c))


b is [1, 2, 3, [4, 5]]
c is [1, 2, 3, [4, 5]]
b is a: False
c is a: False
b is c: False


In [14]:
a == c

True

In [15]:
b == c

True

In [16]:
a[-1] == b[-1]

True

In [17]:
b[-1] == c[-1]

True

### Copies are shallow by default

In [19]:
# 弄清楚浅拷贝深拷贝就行啦

a = [1, 2, 3, [4, 5]]
c = copy.copy(a)
d = copy.deepcopy(a)

a[-1].append(6)

print('a is {}'.format(a))
print('c is {}'.format(c))
print('d is {}'.format(d))

a is [1, 2, 3, [4, 5, 6]]
c is [1, 2, 3, [4, 5, 6]]
d is [1, 2, 3, [4, 5]]


### Function parameters as references


In [22]:
def func(x, y):
    x += y
    return x



In [23]:
x = 1
y = 2
func(x,y)

3

In [24]:
x,y

(1, 2)

In [25]:
x = [1,2]
y = [3,4]
func(x, y)

[1, 2, 3, 4]

In [26]:
x,y

([1, 2, 3, 4], [3, 4])

In [27]:
x = (1, 2)
y = (3, 4)
func(x, y)

(1, 2, 3, 4)

In [28]:
x,y

((1, 2), (3, 4))

### Mutable types as parameter defaults: bad idea


In [29]:
# 面试遇到过的题目

class HauntBus:
    
    def __init__(self, passengers=[]):
        self.passengers = passengers
        
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)
        

In [34]:
# 一般情况
bus1 = HauntBus(['a', 'b'])
bus1.passengers

['a', 'b']

In [32]:
bus1.pick('c')
bus1.passengers

['a', 'b', 'c']

In [33]:
bus1.drop('a')
bus1.passengers

['b', 'c']

In [35]:
# 空车情况
bus2 = HauntBus()
bus2.pick('a')
bus2.passengers

['a']

In [36]:
bus3 = HauntBus()
bus3.pick('c')
bus3.passengers

['a', 'c']

In [37]:
# inspect

dir(HauntBus.__init__)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [39]:
HauntBus.__init__.__defaults__

(['a', 'c'],)

### Defensive programming with mutable parameters

为了防止上述的错误，将可变对象修改为不可变对象

In [40]:
class TwilightBus(HauntBus):
    
    # 重写 init 即可
    def __init__(self, passengers=None):
        if not passengers:
            passengers = []
        self.passengers = passengers
                

In [42]:
bus4 = TwilightBus()
bus4.pick('a')
bus5 = TwilightBus()
bus5.pick('b')
print(bus4.passengers)
print(bus5.passengers)

['a']
['b']


### del and garbage collection


In [43]:
import weakref
s1 = {1, 2, 3}
s2 = s1
def bye():
    print('gone with wind')
    
ender = weakref.finalize(s1, bye)
ender.alive

True

In [44]:
del s1

In [45]:
ender.alive

True

In [46]:
s2 = 'spam'

gone with wind


In [47]:
ender.alive

False

以上例子可得出：
- del 不会删除对象，而是删除引用
- 引用计数的垃圾回收机制，　当 s2 被赋予其他值后，{1,2,3}计数为０，被清除